# Build an LlamaIndex Agent with UnityCatalog functions
In this tutuorial, we'll be covering the steps to use both custom and Databricks-provided AI functions within a LlamaIndex agent. 
We'll be looking at the tracing integrations with MLflow, as well as utilizing MLflow's models-from-code functionality to simplify the logging, registration, and deployment of our Agent that will use UnityCatalog functions as agent tools. 

In [0]:
%pip install -Uqqq unitycatalog-llamaindex[databricks] openai llama_index mlflow

dbutils.library.restartPython()

In [0]:
import base64
import os

from databricks.sdk import WorkspaceClient

workspace_client = WorkspaceClient()

secret_scope = "ben_wilson"  # Change me!

# Run this if you don't have the API key set to your secrets scope yet

# if secret_scope not in [scope.name for scope in workspace_client.secrets.list_scopes()]:
#     workspace_client.secrets.create_scope(secret_scope)

# my_secret = "<your API key, temporarily>"

# workspace_client.secrets.put_secret(scope=secret_scope, key="openai_api_key", string_value=my_secret)

In [0]:
os.environ["OPENAI_API_KEY"] = base64.b64decode(
    workspace_client.secrets.get_secret(scope=secret_scope, key="openai_api_key").value
).decode()

assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

## Define constants and a UnityCatalog client instance

The constants defined below will be used throughout this tutorial with the marked exception in our agent logging script definition. 
For understanding context about the requirements for saving an MLflow model using the [models from code](https://mlflow.org/docs/latest/models.html#models-from-code) feature, carefully read the section that
defines the script that we'll be saving as a model definition.

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

CATALOG = "ben_wilson"  # Change me!
SCHEMA = "uc_func"  # Change me if you want

client = DatabricksFunctionClient()

## Define UC functions

In the next several sections, we will be defining 4 distinct UnityCatalog functions.

- **execute_python_code**: A hand-crafted function using the `create_python_function` API to register custom functionality from a python function
- **ask ai function**: A function that interfaces with the Databricks AI functions services, providing a tool interface for Agents that does not require custom crafting. 
- **summarization function**: A function that interfaces with the Databricks AI function for summarizing text.
- **translation function**: A function that interfaces with the Databricks AI function for providing English <-> Spanish translation without any additional configuration needed.

In [0]:
def execute_python_code(code: str) -> str:
    """
    Executes the given python code and returns its stdout.
    Remember the code should print the final result to stdout.

    Args:
      code: Python code to execute. Remember to print the final result to stdout.
    """
    import sys
    from io import StringIO

    stdout = StringIO()
    sys.stdout = stdout
    exec(code)
    return stdout.getvalue()


function_info = client.create_python_function(
    func=execute_python_code, catalog=CATALOG, schema=SCHEMA, replace=True
)
python_execution_function_name = function_info.full_name

# test execution
client.execute_function(python_execution_function_name, {"code": "print(1+1)"})

FunctionExecutionResult(error=None, format='SCALAR', value='2\n', truncated=None)

In [0]:
ask_ai_function_name = f"{CATALOG}.{SCHEMA}.ask_ai"
sql_body = f"""CREATE OR REPLACE FUNCTION {ask_ai_function_name}(question STRING COMMENT 'A question to submit for confirmation by another language model')
RETURNS STRING
COMMENT 'answer the question by submitting it to the Meta-Llama-3.1-70B-Instruct model'
RETURN SELECT ai_gen(question)
"""
client.create_function(sql_function_body=sql_body)
result = client.execute_function(ask_ai_function_name, {"question": "What is Unity Catalog?"})
result.value

'Unity Catalog is a feature in Databricks that allows you to manage metadata, such as tables, views, and databases, across multiple workspaces and clouds in a single, unified catalog. It provides a centralized way to manage and govern your data assets, making it easier to discover, understand, and use your data.\n\nWith Unity Catalog, you can:\n\n1. **Unify metadata management**: Manage metadata across multiple workspaces, clouds, and regions in a single catalog.\n2. **Simplify data discovery**: Provide a single source of truth for data assets, making it easier for users to find and understand the data they need.\n3. **Improve data governance**: Apply fine-grained access controls and permissions to ensure that sensitive data is protected and only accessible to authorized users.\n4. **Enhance collaboration**: Enable multiple teams and users to work together on data projects, while maintaining control and visibility over data assets.\n\nUnity Catalog provides a range of features, includi

In [0]:
summarization_function_name = f"{CATALOG}.{SCHEMA}.summarize"
sql_body = f"""CREATE OR REPLACE FUNCTION {summarization_function_name}(text STRING COMMENT 'content that is intended to be summarized', max_words INT COMMENT 'The Maximum number of words to generate within the response. The value must be a non-negative integer. If set to 0, then there is no limit to the length of the response.')
RETURNS STRING
COMMENT 'Summarize the content and provide a maximum length to the response that will force varying levels of brevity'
RETURN SELECT ai_summarize(text, max_words)
"""
client.create_function(sql_function_body=sql_body)
# test execution
client.execute_function(summarization_function_name, {"text": result.value, "max_words": 20})

FunctionExecutionResult(error=None, format='SCALAR', value='Unity Catalog: Unified metadata management for Databricks', truncated=None)

In [0]:
translate_function_name = f"{CATALOG}.{SCHEMA}.translate"
sql_body = f"""CREATE OR REPLACE FUNCTION {translate_function_name}(content STRING COMMENT 'Content to translate to the specified language', language STRING COMMENT 'The target language to translate the text to in language shorthand definition. For example, en for English and es for Spanish.')
RETURNS STRING
COMMENT 'Translate the provided content to the specified target language, currently only a bi-directional translation between  English and Spanish is supported.'
RETURN SELECT ai_translate(content, language)
"""
client.create_function(sql_function_body=sql_body)
# test execution
client.execute_function(
    translate_function_name,
    {"content": "What would you like to have for lunch today?", "language": "es"},
)

FunctionExecutionResult(error=None, format='SCALAR', value='¿Qué te gustaría tener para almorzar hoy?</', truncated=None)

### Define the locations of the UC functions

In [0]:
python_execution_function_name = f"{CATALOG}.{SCHEMA}.execute_python_code"
ask_ai_function_name = f"{CATALOG}.{SCHEMA}.ask_ai"
summarization_function_name = f"{CATALOG}.{SCHEMA}.summarize"
translate_function_name = f"{CATALOG}.{SCHEMA}.translate"

## Register Functions as Tools
In order for LlamaIndex to 'understand' what the UnityCatalog function interface is, we need to define our functions as tools in the LlamaIndex format. 

The example below shows how to register all 4 of our functions as LlamaIndex tools, capable of being used within LlamaIndex agents.

In [0]:
# Create tools for LlamaIndex to use

from unitycatalog.ai.llama_index.toolkit import UCFunctionToolkit

toolkit = UCFunctionToolkit(
    client=client,
    function_names=[
        python_execution_function_name,
        ask_ai_function_name,
        summarization_function_name,
        translate_function_name,
    ],
)
tools = toolkit.tools
tools

[UnityCatalogTool(description='Executes the given python code and returns its stdout. Remember the code should print the final result to stdout.', name='ben_wilson__uc_func__execute_python_code', fn_schema=<class 'unitycatalog.ai.core.utils.function_processing_utils.ben_wilson__uc_func__execute_python_code__params'>, return_direct=False),
 UnityCatalogTool(description='answer the question by submitting it to the Meta-Llama-3.1-70B-Instruct model', name='ben_wilson__uc_func__ask_ai', fn_schema=<class 'unitycatalog.ai.core.utils.function_processing_utils.ben_wilson__uc_func__ask_ai__params'>, return_direct=False),
 UnityCatalogTool(description='Summarize the content and provide a maximum length to the response that will force varying levels of brevity', name='ben_wilson__uc_func__summarize', fn_schema=<class 'unitycatalog.ai.core.utils.function_processing_utils.ben_wilson__uc_func__summarize__params'>, return_direct=False),
 UnityCatalogTool(description='Translate the provided content to

## Enable tracing via MLflow

[MLflow's LlamaIndex autologging functionality](https://mlflow.org/docs/latest/llms/llama-index/index.html#enable-tracing) allows for traces to be automatically captured with no code modifications needed. We will gain visibility into our agent's internal processes, including the rationalization involved in tool calling, via the [MLflow Tracing UI](https://mlflow.org/docs/latest/llms/tracing/index.html). 

In [0]:
import mlflow

mlflow.llama_index.autolog()

## Create a LlamaIndex ReActAgent

The Agent we're creating is empowering a configured LLM to utilize our UnityCatalog functions as tools.

In [0]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

llm = OpenAI()

agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

Submit a chat request to the agent and observe the trace that is recorded from tool calls.

In [0]:
agent.chat(
    "Can you provide a succinct summarization of what Databricks UnityCatalog is in less than 50 words and respond in Spanish?"
)

> Running step 1bfe87fe-307c-4646-b2cf-ab2bd9145ad6. Step input: Can you provide a succinct summarization of what Databricks UnityCatalog is in less than 50 words and respond in Spanish?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: ben_wilson__uc_func__translate
Action Input: {'content': 'Databricks Unity Catalog is a centralized governance and security layer for data and analytics in the Databricks Lakehouse Platform.', 'language': 'es'}
Observation: {"format": "SCALAR", "value": "El cat\u00e1logo unificado de Databricks es una capa de gobernanza y seguridad centralizada para datos y an\u00e1lisis en la plataforma Databricks Lakehouse."}
> Running step 4ec6592e-970e-49f5-afe0-0306549ae780. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: El UnityCatalog de Databricks es una plataforma que ofrece activos digitales, herramientas y servicios para el de

AgentChatResponse(response='El UnityCatalog de Databricks es una plataforma que ofrece activos digitales, herramientas y servicios para el desarrollo de juegos, proporcionando modelos 3D, texturas y complementos para agilizar el proceso de desarrollo e integrar activos de manera eficiente.', sources=[ToolOutput(content='{"format": "SCALAR", "value": "El cat\\u00e1logo unificado de Databricks es una capa de gobernanza y seguridad centralizada para datos y an\\u00e1lisis en la plataforma Databricks Lakehouse."}', tool_name='ben_wilson__uc_func__translate', raw_input={'args': (), 'kwargs': {'content': 'Databricks Unity Catalog is a centralized governance and security layer for data and analytics in the Databricks Lakehouse Platform.', 'language': 'es'}}, raw_output='{"format": "SCALAR", "value": "El cat\\u00e1logo unificado de Databricks es una capa de gobernanza y seguridad centralizada para datos y an\\u00e1lisis en la plataforma Databricks Lakehouse."}', is_error=False)], source_nodes=

Trace(request_id=tr-091608d52741416dbbffdc2fbf589729)

## Log the Agent to MLflow

In order to avoid any potential serialization issues when logging our agent, we're going to use the models-from-code feature in MLflow. This logging approach allows for containment of all dependent logic within a Python script that is then executed when loading the model for inference. 

We will define the logic in the next cell, utilizing the Jupyter magic command `%%writefile` at the top of the cell to save a local copy of the cell contents as a python script.

In [0]:
%%writefile agent.py
from mlflow.models import set_model

from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent

from unitycatalog.ai.core.client import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from unitycatalog.ai.llama_index.toolkit import UCFunctionToolkit


# We need to include our constants within the script as it will, upon loading, 
# run in a separate REPL process.
CATALOG = "ben_wilson"
SCHEMA = "uc_func"

# The tool calling functionality will need a UnityCatalog functions client to make tool calls
# to UnityCatalog with the appropriate authorization.
client = DatabricksFunctionClient()

# Define our UC functions pathing
python_execution_function_name = f"{CATALOG}.{SCHEMA}.execute_python_code"
ask_ai_function_name = f"{CATALOG}.{SCHEMA}.ask_ai"
summarization_function_name = f"{CATALOG}.{SCHEMA}.summarize"
translate_function_name = f"{CATALOG}.{SCHEMA}.translate"

# Load our UC functions as tools to be used by LlamaIndex
toolkit = UCFunctionToolkit(
    client=client,
    function_names=[
        python_execution_function_name,
        ask_ai_function_name,
        summarization_function_name,
        translate_function_name,
    ]
)
tools = toolkit.tools

llm = OpenAI()

agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

# In order for MLflow to understand what our invocation function is for the model, 
# we need to specify the interface to the callable as follows:
set_model(agent)

Overwriting agent.py


In [0]:
from mlflow.models import infer_signature

input_example = {
    "message": "Where is the largest population of Grey Wolves in the contiguous United States? Please answer in 6 words or less."
}

# LLamaIndex's Agent interface is a Pydantic model that is not directly supported by MLflow's signature inference.
# To accomodate the output type received from calling the agent, we need to specify what the response will be
# from the invocation (in this case, a string).
signature = infer_signature(input_example, "Northern Rocky Mountains, states like Montana.")

# The pip requirements defined in the model logging call are needed solely due to using a pre-release build of
# `unitycatalog-ai` and the `unitycatalog-llama_index` packages. When using versions available on PyPI, dependency
# inference should work automatically.
with mlflow.start_run():
    model_info = mlflow.llama_index.log_model(
        "agent.py",
        artifact_path="model",
        input_example=input_example,
        signature=signature,
        pip_requirements=[
            "mlflow",
            "unitycatalog-ai",
            "unitycatalog-llamaindex",
            "llama_index",
            "openai",
        ],
        registered_model_name="ben_wilson.uc_func.llama_index_agent",
    )

2024/11/18 17:01:40 INFO mlflow.llama_index.serialize_objects: API key(s) will be removed from the global Settings object during serialization to protect against key leakage. At inference time, the key(s) must be passed as environment variables.


> Running step 9b8079d2-5c8f-41c8-bc0e-f7244a50bc5f. Step input: Where is the largest population of Grey Wolves in the contiguous United States? Please answer in 6 words or less.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: ben_wilson__uc_func__execute_python_code
Action Input: {'code': "print('Yellowstone National Park, Wyoming')"}
Observation: {"format": "SCALAR", "value": "Yellowstone National Park, Wyoming\n"}
> Running step cc53f151-b8f5-4bb7-b04e-f07b74e23b46. Step input: None


2024/11/18 17:01:46 INFO mlflow.models.model: Found the following environment variables used during model inference: [OPENAI_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Yellowstone National Park, Wyoming.


Uploading artifacts:   0%|          | 0/14 [00:00<?, ?it/s]

Registered model 'ben_wilson.uc_func.llama_index_agent' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/14 [00:00<?, ?it/s]

Created version '3' of model 'ben_wilson.uc_func.llama_index_agent'.


🏃 View run agreeable-moth-983 at: https://e2-dogfood.staging.cloud.databricks.com/ml/experiments/1557189199111224/runs/c99078cdef894beda60dfb74e7576137
🧪 View experiment at: https://e2-dogfood.staging.cloud.databricks.com/ml/experiments/1557189199111224


## Validate the ability of our model to be deployed for real-time inference

The validation stage here will ensure that the model that we have logged is capable of being deployed to a model serving endpoint. We're validating that the depenendencies are correct, that our input data structure is viable, and that the agent is capable of responding to requests with the configured tools. 

In [0]:
from mlflow.models import convert_input_example_to_serving_input, validate_serving_input

serving_input = convert_input_example_to_serving_input({"message": "What is 3**10?"})
validate_serving_input(model_info.model_uri, serving_input=serving_input)

> Running step 949962fa-691a-4ee9-a531-91cab4d6eccd. Step input: What is 3**10?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: tool
Action Input: {'code': 'print(3**10)'}
Observation: Error: No such tool named `tool`.
> Running step 5cf8ab1a-c4f6-43c6-82c4-b038a886ada5. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 3 raised to the power of 10 is equal to 59049.


'3 raised to the power of 10 is equal to 59049.'

Trace(request_id=tr-7330fa1860554eda8a220b7e2bac93ac)

## Next Steps
From this point, you can safely deploy your model to a serving endpoint. 

Ensure that the environment variables that were configured at the top of this tutorial (the `OPENAI_API_KEY` and the required Databricks `DATABRICKS_HOST` and `DATABRICKS_TOKEN` variables are set to ensure that the agent can access UnityCatalog for tool calling). 